In [10]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os

In [16]:
data_dir = '../data/test_images'

batch_size = 32
epochs = 8
workers = 0 if os.name == 'nt' else 8

In [17]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


In [18]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

In [20]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
dataset.samples = [
    (p, p.replace(data_dir, data_dir + '_cropped'))
    for p, _ in dataset.samples
]

loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    collate_fn=training.collate_pil
)

for i, (x, y) in enumerate(loader):
    mtcnn(x, save_path=y)
    print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')
del mtcnn

Batch 2 of 2

In [21]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)

In [22]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

dataset = datasets.ImageFolder(data_dir + '_cropped', transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

In [23]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

In [24]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )
writer.close()



Initial
----------
Valid |     1/1    | loss:    0.0000 | fps:    0.9380 | acc:    1.0000   

Epoch 1/8
----------
Train |     1/1    | loss:    0.0000 | fps:    2.6044 | acc:    1.0000   
Valid |     1/1    | loss:    0.0000 | fps:    1.0535 | acc:    1.0000   

Epoch 2/8
----------
Train |     1/1    | loss:    0.0000 | fps:    2.7351 | acc:    1.0000   
Valid |     1/1    | loss:    0.0000 | fps:    1.0685 | acc:    1.0000   

Epoch 3/8
----------
Train |     1/1    | loss:    0.0000 | fps:    2.8422 | acc:    1.0000   
Valid |     1/1    | loss:    0.0000 | fps:    1.0602 | acc:    1.0000   

Epoch 4/8
----------
Train |     1/1    | loss:    0.0000 | fps:    2.8456 | acc:    1.0000   
Valid |     1/1    | loss:    0.0000 | fps:    1.0713 | acc:    1.0000   

Epoch 5/8
----------
Train |     1/1    | loss:    0.0000 | fps:    2.8344 | acc:    1.0000   
Valid |     1/1    | loss:    0.0000 | fps:    1.0801 | acc:    1.0000   

Epoch 6/8
----------
Train |     1/1    | loss:    0.0

In [25]:
torch.save(resnet.state_dict(), 'model_weights/model.pth')